# Machine Translation

In diesem Notebook möchten wir uns mit der Königsdisziplin des Natural Language Processings beschäftigen, der maschinellen Übersetzung.
Vermutlich gibt es keine NLP-Anwendung, die einerseits vielen bekannt ist, anderseits durch Deep Learning und neuronalen Netzen einen solchen Aufschwung bekommen hat.
Vor dem Siegeszug der neuronalen Netzen, wurde maschinelle Übersetzung deshalb als schwierig angesehen, weil sie alle Teilaspekte von Sprache beinhaltet. Neben grammatikalischer Korrektheit, sollen maschinell übersetzte Texte den Sinn des Originaltextes wiedergeben und darüberhinaus auch den Subtext, wie Ironie, Witz, erfassen.
Ob letzteres einfach gelingt, sei hier mal dahingestellt.

## Daten
Im Vergleich zu den bisherigen Notebooks sind die Daten für machine translation etwas dröge. Daher möchten wir uns hier nicht allzu lange aufhalten. Speichert zunächst folgende Datei ab und entpackt sie: http://www.manythings.org/anki/fra-eng.zip

Danach wollen wir die Daten einlesen. Das Format ist einfach, pro Zeile steht ein Satzpaar getrennt von einem Tab.

In [1]:
with open('fra.txt', 'r') as f:
    lines = f.readlines()

In [2]:
pairs = [tuple(l.split("\t")) for l in lines]

In [3]:
pairs_cleaned = [(e, f.strip()) for e, f in pairs]

Damit unser Modell weiß wo Anfang und Ende der französischen Sätze sind, müssen wir noch ein spezielles Start- (🏳️) und Endsymbol (🏴) einfügen:

In [4]:
pairs_cleaned_with_marker = [(e, "🏳️" + f + "🏴") for e, f in pairs_cleaned]

Speichert euch nun alle Character, die in den englischen Sätzen und in den französischen Sätzen in jeweils einem Set ab.

In [5]:
print(pairs_cleaned_with_marker[0])

('Go.', '🏳️Va !🏴')


In [6]:
english_characters = set()
french_characters = set()

for str_en, str_fr in pairs_cleaned_with_marker:
    [english_characters.add(char) for char in str_en]
    [french_characters.add(char) for char in str_fr]

english_characters = list(english_characters)
french_characters = list(french_characters)

print(english_characters)
print(french_characters)

['/', '"', '%', ';', '&', '’', 'N', '+', '1', 's', 'ú', 'e', 'D', "'", '7', 'f', '!', 'o', 'w', '4', ' ', '5', '$', 'W', '-', 'ö', 'n', 'q', 'ç', 'H', '‘', '?', 'S', 'c', 'm', 'G', 'T', '€', '2', 'Q', 'r', 'J', 'b', 'Y', 't', 'L', 'i', 'k', '9', '—', 'x', '0', 'Z', ':', 'X', 'z', 'E', 'V', 'p', 'M', 'A', 'I', 'C', '3', '₂', 'а', 'º', 'a', 'y', 'B', 'R', 'u', 'P', '.', 'F', 'é', 'd', '6', 'v', 'g', 'O', '\xad', '8', 'j', 'U', ',', 'l', 'K', '\xa0', 'h', '–']
['/', '"', '%', '️', ';', '‽', '«', 'Ê', '’', '&', 'N', 'ù', ')', '+', '1', 's', '🏴', 'Ç', 'e', 'D', '»', "'", '7', 'f', '!', 'o', '\u200b', 'w', '4', ' ', '5', '$', 'W', '-', '…', 'ö', 'n', 'q', 'ç', 'H', '‘', '?', 'S', 'c', 'm', 'G', 'T', 'î', 'ô', 'Q', '2', 'r', 'J', 'b', 'É', 'Y', 't', 'L', 'i', 'k', '9', 'ë', 'x', '0', 'Z', 'С', ':', '\u2009', 'X', 'z', 'ï', 'â', '\u202f', 'E', 'V', 'p', 'M', '(', 'A', 'I', 'C', '3', '₂', 'a', 'y', 'À', 'œ', 'B', 'R', 'u', 'P', '.', 'F', 'û', 'Ô', 'é', 'd', '6', 'v', 'à', 'g', 'O', '8', 'è', 'j

Um unser Modell später mit passenden Eingabedaten füttern zu können, müssen wir nun noch die jeweiligen Vektorlängen für den encoder und den decoder bestimmen. Speichert die Längen in den Variablen  `encoder_vector_len` und `decoder_vector_len` ab. Hinweis: uns Modell bekommt Vektoren, die die Characters pro Wort abbilden als Eingabe.

In [7]:
encoder_vector_len = len(english_characters)#TODO
decoder_vector_len = len(french_characters)#TODO
print(encoder_vector_len)
print(decoder_vector_len)

91
116


Um die Eingabevektoren bestimmen zu können, müssen wir uns noch Dictionaries bauen, die jeweils die Character auf die Komponenten des Vektors abbilden

In [8]:

english_token_index = {char:key for key, char in enumerate(english_characters)}
french_token_index = {char:key for key, char in enumerate(french_characters)}

print(english_token_index) #token2idx
print(french_token_index)

{'x': 50, '0': 51, '/': 0, '"': 1, '%': 2, 'r': 40, 'Z': 52, ';': 3, ':': 53, 'X': 54, '&': 4, 'N': 6, 'E': 56, '+': 7, 's': 9, '1': 8, 'V': 57, 'p': 58, 'W': 23, 'M': 59, 'ú': 10, 'I': 61, '3': 63, '₂': 64, 'k': 47, 'e': 11, 'а': 65, 'D': 12, "'": 13, '7': 14, '2': 38, 'f': 15, 'g': 79, '!': 16, 'o': 17, 'l': 86, 'w': 18, 'B': 69, 'q': 27, '4': 19, 'R': 70, ' ': 20, 'u': 71, 'P': 72, '.': 73, '-': 24, 'F': 74, 'ö': 25, 'é': 75, 'd': 76, '’': 5, 'n': 26, 'C': 62, '6': 77, 'ç': 28, 'v': 78, 'H': 29, '‘': 30, '?': 31, 'S': 32, 'm': 34, 'O': 80, '5': 21, 'G': 35, '\xad': 81, '8': 82, 'z': 55, 'T': 36, 'j': 83, '€': 37, 'U': 84, ',': 85, 'Q': 39, 'b': 42, 'J': 41, 'a': 67, '$': 22, 'A': 60, 'K': 87, 'Y': 43, 'º': 66, 't': 44, 'L': 45, 'i': 46, '—': 49, '9': 48, '\xa0': 88, 'h': 89, 'c': 33, 'y': 68, '–': 90}
{'x': 62, '0': 63, '/': 0, '"': 1, '%': 2, 'b': 53, 'Z': 64, 'С': 65, ';': 4, ':': 66, '‽': 5, '«': 6, 'Ê': 7, '\u2009': 67, 'F': 92, 'X': 68, '’': 8, ')': 12, 'ù': 11, 'ï': 70, 'â': 7

Wir haben nun alle Dimensionen bestimmt, um die Trainingsdaten zu definieren. Dafür bauen wir uns drei Matrizen (eine als Eingabe für den Encoder, eine als Eingabe für den Decoder und eine als Ausgabe des Decoders) zusammen. Die Dimensionen dafür sind jeweils: Anzahl Sätze, Anzahl Wörter des längsten Satzes, Anzahl Characters.

In [ ]:
import numpy as np

max_encoder_sentence_len = max((len(e) for e, _ in pairs_cleaned_with_marker))
max_decoder_sentence_len = max((len(f) for _, f in pairs_cleaned_with_marker))


encoder_input_data = np.zeros(
    (len(pairs_cleaned_with_marker), max_encoder_sentence_len, encoder_vector_len),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(pairs_cleaned_with_marker), max_decoder_sentence_len, decoder_vector_len),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(pairs_cleaned_with_marker), max_decoder_sentence_len, decoder_vector_len),
    dtype='float32')

Befüllt nun die Matrizen. Die `encoder_input_data` und die `decoder_input_data`-Matrix soll für jeden Satz aus den Trainingsdaten, einen Eintrag mit einem Token-Vektor pro Wort enthalten.
Die `decoder_target_data`-Matrix soll den gleichen Inhalt wie die `decoder_input_data`-Matrix haben, allerdings um eine Position verschoben. (Schaut euch nochmal die Folien an, wir brauchen Platz für das Startsymbol) 

Die `encoder_input_data`-Matrix wird mit den englischen Daten befüllt, die beiden anderen mit den französischen Daten.



In [ ]:
for i, (e, f) in enumerate(pairs_cleaned_with_marker):
    for t, char in enumerate(e):
        encoder_input_data[i, t, english_token_index[char]] = 1.
    for t, char in enumerate(f):
        decoder_input_data[i, t, french_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i, t-1, french_token_index[char]] = 1.

## Sequence-to-sequence Modelle

Im Gegensatz zu den bisherigen Notebooks, wollen wir hier mit dem Modell beginnen. Einerseits, weil die Daten eher dröge  (trocken) sind (englisch-fränzosische Satzpaare), andererseits, weil das Modell selbst komplizierter ist und wir hier die Functional-API von Keras benutzen müssen.

Im Gegensatz zu der Sequence-API, definiert man hier ein Modell in dem, den jeweiligen Layer mit dem Vorgänger Layer aufruft:
```

a = Input(...)
b = Dense(...)

t = b(a)


```
Als Rückgabe bekommt man einen Tensor.


Ganz generell gesprochen, möchten wir hier ein Seq2Seq-Modell aufbauen, das auf LSTMs basiert. Wie auf den Folien beschrieben wurde, teilt sich so ein Modell in einen Encoder und Decoder auf. 
Wir beginnen die Implementierung mit dem Encoder.

### Encoder

Auch das komplexeste Modell fängt mit einer Eingabe ein.

Definiert einen Input-Layer, der als Eingabedimension, Sätze beliebiger Länge als Vektoren nimmt. Eine Komponente im Vektor repräsentiert einen Character. Unbekannte Teile der Eingabedimensionen kann man mit `None` definieren.

In [ ]:
from keras.layers import Input

encoder_input = Input(shape=(None, encoder_vector_len))

Using TensorFlow backend.


Als nächstes definiert einen LSTM-Layer, der 256 hidden units hat. Die Ausgabe des Layers soll später an den Decoder 'verfüttert' werden. Seht in der Dokumentation nach wie man den Thought-Vektor zurückbekommen kann (https://keras.io/layers/recurrent/)

In [ ]:
from keras.layers import LSTM
lstm_hu = 256
encoder = LSTM(units=lstm_hu, return_state=True)

Verbindet nun den `encoder_input`-Layer mit dem `encoder`-Layer

In [ ]:
_, state_h, state_c = encoder(encoder_input)

Instructions for updating:
Colocations handled automatically by placer.


Der Encoder wäre damit fertig implementiert.

### Decoder

Auch der Decoder fängt mit einem Input-Layer an. Definiert einen Input-Layer analog zum Encoder.

In [ ]:
decoder_input = Input(shape=(None, decoder_vector_len))

Wie auch im Encoder, ist das Herzstück ein LSTM-Layer. Definiert einen LSTM-Layer mit 256 hidden units. Seht in der Dokumentation nach, wie man neben den Thought-Vektoren, auch die komplette Ausgabe-Sequence zurückbekommt.

In [ ]:
decoder_lstm = LSTM(256, return_state=True, return_sequences=True)

Verbindet nun den `decoder_input`-Layer mit dem `decoder_lstm`-Layer. An welcher der drei Ausgaben sind wir hier interessiert? Außerdem müssen wir hier, den Thought-Vektor des Encoders mit übergeben, dies geschieht über den `initial_state`-Parameter

In [ ]:
sequences ,_,_ = decoder_lstm(decoder_input, initial_state=[state_h,state_c])

Den Abschluss unseres Modells bildet ein Dense-Layer, der gleich viele `hidden_units` wie der Input-Layer des Decoders hat. Als Aktivierungsfunktion nehmen wir die Softmax-Funktion.

In [ ]:
from keras.layers import Dense

decoder_layer = Dense(decoder_vector_len, activation="softmax")

Verbindet diesen Layer mit den bisherigen Decoder sequences.

In [ ]:
decoder = decoder_layer(sequences)

Wir haben nun alle Elemente für ein Sequence-To-Sequence-Modell zusammen. Im letzten Schritt bauen wir das ganze Modell zusammen.

In [ ]:
from keras import Model

model = Model(inputs=[encoder_input, decoder_input], outputs=[decoder])
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

Fertig! Nun kann das Modell trainiert werden.

In [ ]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=256,
          epochs=3,
          validation_split=0.2)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 133704 samples, validate on 33426 samples
Epoch 1/3
133704/133704 [==============================] - 419s 3ms/step - loss: 0.2095 - val_loss: 0.3263
Epoch 2/3
 17152/133704 [==>...........................] - ETA: 5:18 - loss: 0.1622

### Inferenz

Um mit dem trainierten Modell Texte übersetzen zu können, müssen wir nun noch unser bisheriges Modell etwas umbauen.
Im ersten Schritt definieren wir uns ein `Model` das einen Eingabetext encodiert. Als Eingabe hat dieses `Model` den bisherigen `encoder_input`-Layer und die Thought-Vektoren als Output. 


In [ ]:
encoder_model = Model(encoder_input, [state_h, state_c])

In [ ]:
encoder_model.summary()

Etwas komplizierter ist das Decoder-Modell. Als Input nimmt es zum einen die Thought-Vektoren des Encoders, zum anderen den `decoder_input`-Layer von oben.
Definiert zunächst zwei Input-Layer für die Thought-Vektoren.

In [ ]:
decoder_state_input_h = Input(shape=(lstm_hu,))
decoder_state_input_c = Input(shape=(lstm_hu,))

Setzt diese beiden Input-Layer als `initial_state` und den `decoder_input`-Layer als Input in das `decoder_lstm` von oben ein.

In [ ]:
decoder_sequences, state_h, state_c = decoder_lstm(decoder_input, initial_state=[decoder_state_input_h, decoder_state_input_c])

Setzt nun den `decoder_outputs`-Tensor in den `decoder_layer` von oben ein.

In [ ]:
decoder_outputs = decoder_layer(decoder_sequences)

Nun können wir ein `Model` zusammenbauen, das uns als Decoder dient. Input ist hier der `decoder_input`-Layer, zusammen mit den `decoder_state_input_h` und dem `decoder_state_input_c`-Tensor. Als Output dient uns der `decoder_outputs`-Tensor und der `state_h` mit dem `state_c`-Tensor. 

In [ ]:
decoder_model = Model(inputs=[decoder_input, decoder_state_input_h, decoder_state_input_c], outputs=[decoder_outputs, state_h, state_c])

Ein kleiner Zwischenschritt ist noch nötig, bevor wir mit dem Modell Texte übersetzen können. Damit wir aus unseren Vektoren wieder Texte bekommen, müssen wir noch das Dictionary `reverse_french_token_index` befüllen, das die "Umkehrung" von `french_token_index` ist.

In [ ]:
reverse_french_token_index = {idx:token for token, idx in french_token_index.items()} #idx2token
reverse_english_token_index = {idx:token for token, idx in english_token_index.items()} #idx2token

In [ ]:
print(reverse_french_token_index)

Nun können wir eine Funktion `translate` implementieren, die als Eingabe einen englischen Text als Vektoren nimmt.

In [ ]:

#first start: predict_char(french_token_index["🏳"], thought[0], thought[1]) -> french_char_idx, h, c
#every iter: predict_char(french_char_idx, h, c) -> french_char_idx, h, c


def predict_char(prev_char_idx, state_h, state_c):
    french_char_onehot = np.zeros((1, 1, decoder_vector_len))
    french_char_onehot[0, 0, prev_char_idx] = 1
    predicted_chars_probs, h, c = decoder_model.predict([french_char_onehot, state_h, state_c])
    predicted_french_char_idx = np.argmax(predicted_chars_probs)
    predicted_french_char = reverse_french_token_index[predicted_french_char_idx]
    return predicted_french_char_idx, predicted_french_char, h, c

def translate(english_text):
    # Wandelt mit dem encoder_model, `english_text` in einen Thought-Vektor um
    thought = encoder_model.predict(english_text)
    
    translation = ""
    
    predicted_french_char_idx, h, c = french_token_index["🏳"], thought[0], thought[1]
    
    while predicted_french_char_idx != french_token_index["🏴"]:
        predicted_french_char_idx, predicted_french_char, h, c = predict_char(predicted_french_char_idx, h, c)
        #print(h[0][0])
        translation += predicted_french_char
    
    return translation[:-1]

    """
    ----------
    # Als Input für den Decoder, wird zum einen der thought-Vektor benötigt, 
    # zum anderen die Vektor-Repräsentation des Startsymbols
    french_text = np.zeros((1, 1, decoder_vector_len)) #TODO
    
    # Belegt die Komponente des Start-Symbols mit 1
    french_text[0, 0, french_token_index["🏳"]] = 1.  #TODO
    
    # und übergebt beides an den Decoder
    chars, h, c = decoder_model.predict([french_text] + thought)
    print(chars)
    # chars beinhaltet eine 1 x 1 x decoder_vector_len Matrix, die für jeden französische Character
    # eine Art "Wahrscheinlichkeit" angibt.
    # Findet die Komponente, mit der größten Wahrscheinlichkeit und wandelt sie in ein Character um
    french_char_idx = np.argmax(chars) #TODO
    french_char = reverse_french_token_index[french_char_idx]
    print(french_char_idx, french_char)
    
    -------------------
    # die nächste Prediction wäre dann
    french_text = np.zeros((1, 1, decoder_vector_len))
    french_text[0, 0, french_char_idx] = 1.
    chars, h, c = decoder_model.predict([french_text, h, c])
    french_char_idx = np.argmax(chars) #TODO
    french_char = reverse_french_token_index[french_char_idx]
    print(french_char_idx, french_char)
    -----------------
    """
    # TODO: baut eine Schleife mit geeigneter Abbruchbedingung, die die Schritte von oben umfasst 
    # und dabei aus french_char einen übersetzen Text zusammenbaut

In [ ]:
english_text = np.array([encoder_input_data[10]])
print(english_text.shape, len(english_text[0]), encoder_vector_len)

french_text = translate(english_text)

print("-- English --")
#print(pairs_cleaned_with_marker[10])
print("".join([reverse_english_token_index[np.argmax(char)] for char in english_text[0]]))
print("-- French --")
print(french_text)


In [ ]:
print(pairs_cleaned_with_marker[0:3])

In [ ]:
#TODO: save model to disc